### Step 1 – Extract

In [92]:

!pip install faker

In [93]:

# STEP 1: EXTRACT DATA
# Import required libraries
import pandas as pd        # For handling and manipulating tabular data
import numpy as np         # For generating random numbers and arrays
from faker import Faker    # For creating fake but realistic-looking data
from datetime import datetime, timedelta  # For working with dates

def generate_synthetic_data(num_rows=1000):
    """
    Generates synthetic retail sales data using Faker and NumPy.
    The data structure mimics the UCI Online Retail dataset.

    Parameters:
    ----------
    num_rows : int
        Number of rows (records) to generate. Default = 1000.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the generated synthetic sales data.
    """

    # Initialize Faker object for creating fake values
    fake = Faker()

    # Container to store generated rows
    data = []

    # Define the start and end dates for random date generation
    start_date = datetime(2023, 8, 12)
    end_date = datetime(2025, 8, 12)

    # Calculate total number of days in the range for date randomization
    date_range = (end_date - start_date).days

    # Loop to generate each record
    for _ in range(num_rows):
        # Generate random invoice number (5 digits)
        invoice_no = fake.random_int(min=10000, max=99999)

        # Generate random stock code (two letters + three digits)
        stock_code = fake.bothify(text='??###')

        # Create a fake product description by combining two words
        description = fake.word().capitalize() + " " + fake.word().capitalize()

        # Generate a quantity between 1 and 50
        quantity = np.random.randint(1, 50)

        # Generate a random date within the given range
        invoice_date = start_date + timedelta(days=np.random.randint(0, date_range))

        # Generate a unit price between 1 and 100 (rounded to 2 decimal places)
        unit_price = round(np.random.uniform(1, 100), 2)

        # Random customer ID between 1 and 100 (100 unique customers)
        customer_id = np.random.randint(1, 101)

        # Generate a random country name
        country = fake.country()

        # Append the generated record to the list
        data.append([invoice_no, stock_code, description, quantity, invoice_date, unit_price, customer_id, country])

    # Define column names similar to the original Online Retail dataset
    columns = ["InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country",]

    # Convert the list of records into a pandas DataFrame
    return pd.DataFrame(data, columns=columns)

# Generate and preview the synthetic dataset
df = generate_synthetic_data()

# Display the first 5 rows to verify data structure
print(df.head())


   InvoiceNo StockCode       Description  Quantity InvoiceDate  UnitPrice  \
0      40330     jE690   Prevent Station         7  2024-09-21      79.37   
1      84535     dh830  Conference Short         5  2025-04-06      55.73   
2      28647     TR953        Lead Stock        34  2023-09-07      44.10   
3      12264     kh598       Section War        44  2023-09-16      80.80   
4      38327     AN786        Small Save        18  2024-10-22      52.04   

   CustomerID                                            Country  
0          34                                            Bolivia  
1          59                                         Cape Verde  
2          42  British Indian Ocean Territory (Chagos Archipe...  
3          64                               Netherlands Antilles  
4          17                                 Russian Federation  


In [94]:
 #  Save the generated dataset to CSV 
df.to_csv("synthetic_retail_data.csv", index=False)

### Step 2 – Transform
### Clean, calculate new fields, filter, and create dimension-like summaries.

In [95]:

# STEP 2: TRANSFORM DATA

# Ensure InvoiceDate is datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Add TotalSales column
df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

# Remove outliers
df = df[(df["Quantity"] >= 0) & (df["UnitPrice"] > 0)]

# Filter last year (Aug 12, 2024 to Aug 12, 2025)
filter_start = datetime(2024, 8, 12)
filter_end = datetime(2025, 8, 12)
df_last_year = df[(df["InvoiceDate"] >= filter_start) & (df["InvoiceDate"] <= filter_end)]

# Create Customer summary
customer_summary = df_last_year.groupby("CustomerID").agg({
    "TotalSales": "sum",
    "Country": "first"
}).reset_index()

# Create Time dimension table
time_dim = df_last_year[["InvoiceDate"]].drop_duplicates().copy()
time_dim["Year"] = time_dim["InvoiceDate"].dt.year
time_dim["Month"] = time_dim["InvoiceDate"].dt.month
time_dim["Quarter"] = time_dim["InvoiceDate"].dt.quarter

print("Transformed Sales Data:\n", df_last_year.head())
print("\nCustomer Summary:\n", customer_summary.head())
print("\nTime Dimension:\n", time_dim.head())


Transformed Sales Data:
    InvoiceNo StockCode       Description  Quantity InvoiceDate  UnitPrice  \
0      40330     jE690   Prevent Station         7  2024-09-21      79.37   
1      84535     dh830  Conference Short         5  2025-04-06      55.73   
4      38327     AN786        Small Save        18  2024-10-22      52.04   
5      50479     NB552         Them Grow         1  2025-07-07      21.27   
7      70186     Db875      Project Test        26  2025-07-17      11.92   

   CustomerID             Country  TotalSales  
0          34             Bolivia      555.59  
1          59          Cape Verde      278.65  
4          17  Russian Federation      936.72  
5          23         Isle of Man       21.27  
7         100        Saint Helena      309.92  

Customer Summary:
    CustomerID  TotalSales      Country
0           1     4193.23         Oman
1           2     2981.42         Oman
2           3    11521.61   Mauritania
3           4     8097.38  Philippines
4        

### Step 3 – Load
 Store the transformed data into SQLite database (retail_dw.db).

In [96]:

# STEP 3: LOAD DATA


import sqlite3

# Connect to SQLite database (creates file if it doesn't exist)
conn = sqlite3.connect("retail_dw.db")

# Load tables into database
df_last_year.to_sql("SalesFact", conn, if_exists="replace", index=False)
customer_summary.to_sql("CustomerDim", conn, if_exists="replace", index=False)
time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)

# Close connection
conn.close()

print("Data successfully loaded into retail_dw.db")


Data successfully loaded into retail_dw.db


In [97]:
### Step 4 – Combine into ETL Function


In [ ]:
def run_etl():

    # EXTRACT PHASE
   
    print("🔹 Extracting...")
    # Generate synthetic retail data (mimics UCI Online Retail dataset)
    df = generate_synthetic_data()
    print(f"Rows Extracted: {len(df)}")  # Log total rows extracted

    # TRANSFORM PHASE
  
    print("🔹 Transforming...")

    # Convert InvoiceDate column to datetime for proper filtering and grouping
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

    # Calculate TotalSales as Quantity × UnitPrice
    df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

    # Remove invalid rows where Quantity is negative or UnitPrice is zero/negative
    df = df[(df["Quantity"] >= 0) & (df["UnitPrice"] > 0)]

    # Define date range for "last year" sales (exam's reference date: Aug 12, 2025)
    filter_start = datetime(2024, 8, 12)
    filter_end = datetime(2025, 8, 12)

    # Filter dataset for only the sales in the last year
    df_last_year = df[(df["InvoiceDate"] >= filter_start) & (df["InvoiceDate"] <= filter_end)]

    # Create a Customer Dimension table:
    # - Group by CustomerID
    # - Aggregate total sales per customer
    # - Keep the first encountered country (assuming one per customer)
    customer_summary = df_last_year.groupby("CustomerID").agg({
        "TotalSales": "sum",
        "Country": "first"
    }).reset_index()

    # Create a Time Dimension table:
    # - Unique dates from last year's sales
    # - Extract Year, Month, and Quarter for analysis
    time_dim = df_last_year[["InvoiceDate"]].drop_duplicates().copy()
    time_dim["Year"] = time_dim["InvoiceDate"].dt.year
    time_dim["Month"] = time_dim["InvoiceDate"].dt.month
    time_dim["Quarter"] = time_dim["InvoiceDate"].dt.quarter

    print(f"Rows after transform: {len(df_last_year)}")  # Log rows after filtering

    # LOAD PHASE
   
    print("🔹 Loading into DB...")

    # Connect to (or create) SQLite database file retail_dw.db
    conn = sqlite3.connect("retail_dw.db")

    # Load fact table: SalesFact (last year's cleaned sales data)
    df_last_year.to_sql("SalesFact", conn, if_exists="replace", index=False)

    # Load dimension table: CustomerDim
    customer_summary.to_sql("CustomerDim", conn, if_exists="replace", index=False)

    # Load dimension table: TimeDim
    time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)

    # Close database connection
    conn.close()

    print(" ETL Complete!")  # Log completion message


# Run full ETL process
run_etl()


🔹 Extracting...
Rows Extracted: 1000
🔹 Transforming...
Rows after transform: 510
🔹 Loading into DB...
 ETL Complete!


### Step 4: the full ETL function

In [99]:
import sqlite3  # For database connection

def run_etl():
    """
    Runs the full ETL (Extract, Transform, Load) process for the synthetic retail dataset.
    Logs the number of rows processed at each stage.
    """


    # EXTRACT
  
    print("🔹 STEP 1: Extracting data...")
    df = generate_synthetic_data()  # Call our extraction function
    print(f" Extracted {len(df)} rows.")

   
    # TRANSFORM
 
    print("🔹 STEP 2: Transforming data...")

    # Convert InvoiceDate to datetime type
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

    # Create a new column for total sales value
    df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

    # Remove invalid rows: Quantity < 0 or UnitPrice <= 0
    df = df[(df["Quantity"] >= 0) & (df["UnitPrice"] > 0)]

    # Filter for the last year (from 2024-08-12 to 2025-08-12)
    filter_start = datetime(2024, 8, 12)
    filter_end = datetime(2025, 8, 12)
    df_last_year = df[(df["InvoiceDate"] >= filter_start) & (df["InvoiceDate"] <= filter_end)]

    # Create Customer Dimension (summary)
    customer_summary = df_last_year.groupby("CustomerID").agg({
        "TotalSales": "sum",
        "Country": "first"  # Keep first country per customer
    }).reset_index()

    # Create Time Dimension (unique dates with year, month, quarter)
    time_dim = df_last_year[["InvoiceDate"]].drop_duplicates().copy()
    time_dim["Year"] = time_dim["InvoiceDate"].dt.year
    time_dim["Month"] = time_dim["InvoiceDate"].dt.month
    time_dim["Quarter"] = time_dim["InvoiceDate"].dt.quarter

    print(f" After transformation: {len(df_last_year)} rows in fact table.")


    # LOAD
  
    print("🔹 STEP 3: Loading data into SQLite database...")
    conn = sqlite3.connect("retail_dw.db")  # Create DB file
    df_last_year.to_sql("SalesFact", conn, if_exists="replace", index=False)  # Fact table
    customer_summary.to_sql("CustomerDim", conn, if_exists="replace", index=False)  # Customer dimension
    time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)  # Time dimension
    conn.close()
    print("Data successfully loaded into retail_dw.db")


    # LOG SUMMARY
    
    print("\n ETL PROCESS SUMMARY")
    print(f" - Rows Extracted: {len(df)}")
    print(f" - Rows After Transformation (Fact Table): {len(df_last_year)}")
    print(f" - CustomerDim Rows: {len(customer_summary)}")
    print(f" - TimeDim Rows: {len(time_dim)}")
    print(" ETL Completed Successfully!")

# Run the ETL process
run_etl()


🔹 STEP 1: Extracting data...
 Extracted 1000 rows.
🔹 STEP 2: Transforming data...
 After transformation: 505 rows in fact table.
🔹 STEP 3: Loading data into SQLite database...
Data successfully loaded into retail_dw.db

 ETL PROCESS SUMMARY
 - Rows Extracted: 1000
 - Rows After Transformation (Fact Table): 505
 - CustomerDim Rows: 100
 - TimeDim Rows: 276
 ETL Completed Successfully!


### Task 3: OLAP Queries
1. Roll-up: Total Sales by Country and Quarter


In [100]:
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("retail_dw.db")

with sqlite3.connect("retail_dw.db") as conn:
    # Define SQL query to aggregate sales by country and quarter
    query_rollup_fixed = """
    SELECT 
        sf.Country,               -- Select country from SalesFact table
        td.Quarter,               -- Select quarter from TimeDim table
        SUM(sf.TotalSales) AS TotalSales  -- Calculate total sales for each group
    FROM SalesFact sf
    JOIN TimeDim td ON sf.InvoiceDate = td.InvoiceDate  -- Join on InvoiceDate to get quarter info
    GROUP BY sf.Country, td.Quarter                    -- Group results by country and quarter
    ORDER BY sf.Country, td.Quarter;                   -- Sort output by country and quarter
    """
    # Execute the query and load results into a DataFrame
    rollup_df = pd.read_sql_query(query_rollup_fixed, conn)
    # Print the roll-up results
    print("\n--- Roll-up: Total sales by Country & Quarter ---")
    print(rollup_df)


--- Roll-up: Total sales by Country & Quarter ---
               Country  Quarter  TotalSales
0          Afghanistan        1     1098.30
1          Afghanistan        2     1310.36
2              Albania        4      591.63
3              Algeria        2       69.69
4              Algeria        3      664.09
..                 ...      ...         ...
386  Wallis and Futuna        1      285.48
387  Wallis and Futuna        4      775.68
388              Yemen        3     1076.09
389           Zimbabwe        1     1597.32
390           Zimbabwe        4     1949.60

[391 rows x 3 columns]


 2️, Drill-down: Monthly Sales for a Specific Country

In [101]:
# Connect to the SQLite database using the existing connection object 'conn'
# (conn is already defined in the notebook and points to 'retail_dw.db')

# Define the SQL query for drill-down: monthly sales for the selected country (country_name)
# This query:
# - Selects Year and Month from the TimeDim table
# - Sums TotalSales from the SalesFact table for each month
# - Joins SalesFact and TimeDim on InvoiceDate to get month/year info
# - Filters for rows where SalesFact.Country matches the given country_name
# - Groups results by Year and Month
# - Orders results by Year and Month for chronological output
query_drilldown_fixed = """
    SELECT td.Year,                    -- Extract year from TimeDim
           td.Month,                   -- Extract month from TimeDim
           SUM(sf.TotalSales) AS MonthlySales  -- Aggregate total sales for the month
    FROM SalesFact sf                  -- Use SalesFact as main table
    JOIN TimeDim td ON sf.InvoiceDate = td.InvoiceDate  -- Join to get time attributes
    WHERE sf.Country = ?               -- Filter for the selected country
    GROUP BY td.Year, td.Month         -- Group by year and month
    ORDER BY td.Year, td.Month;        -- Sort results chronologically
"""

# Execute the SQL query using pandas read_sql_query
# - Pass the query string
# - Use the existing connection 'conn'
# - Provide the country_name as a parameter to the query (for WHERE clause)
drilldown_df = pd.read_sql_query(query_drilldown_fixed, conn, params=(country_name,))

# Print a header to indicate the drill-down results for the selected country
print(f"\n--- Drill-down: Monthly sales for {country_name} ---")

# Display the resulting DataFrame containing monthly sales
print(drilldown_df)


--- Drill-down: Monthly sales for Belarus ---
Empty DataFrame
Columns: [Year, Month, MonthlySales]
Index: []


3️, Slice: Total Sales by Category (focus on Electronics)

In [102]:
# OLAP Slice Query: Top 10 Product Descriptions by Total Sales

# The following code executes an OLAP "slice" query to find the top 10 product descriptions
# with the highest total sales in the SalesFact table of the retail_dw.db SQLite database.

# The connection object 'conn' is already available in the notebook.

# Define the SQL query:
# - Selects the Description column (product description)
# - Sums the TotalSales for each description
# - Groups results by Description
# - Orders by total sales in descending order
# - Limits output to top 10 descriptions
query_slice_fixed = """
    SELECT Description,                      -- Product description
           SUM(TotalSales) AS DescriptionSales  -- Total sales for each description
    FROM SalesFact
    GROUP BY Description                      -- Group by product description
    ORDER BY DescriptionSales DESC            -- Sort by sales descending
    LIMIT 10;                                 -- Only top 10 results
    """

# Execute the query using pandas read_sql_query:
# - Pass the query string
# - Use the existing SQLite connection 'conn'
slice_df = pd.read_sql_query(query_slice_fixed, conn)

# Print a header for clarity
print("\n--- Slice: Top 10 product descriptions by total sales ---")

# Display the resulting DataFrame
print(slice_df)


--- Slice: Top 10 product descriptions by total sales ---
       Description  DescriptionSales
0     College Blue           4837.77
1    Eight Himself           4658.64
2       Have Party           4632.96
3       East Field           4577.76
4  Measure Develop           4486.15
5       Think Lead           4236.60
6      Create Loss           4231.35
7   Measure Policy           4200.24
8        Oil Piece           4126.13
9   Artist Evening           4015.68


 Plot: Bar chart of total sales by country

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to your SQLite database
conn = sqlite3.connect("retail_dw.db")

# Query: Total sales by country
query_country_sales = """
SELECT c.Country,
       SUM(s.TotalSales) AS TotalSales
FROM SalesFact s
JOIN CustomerDim c ON s.CustomerID = c.CustomerID
GROUP BY c.Country
ORDER BY TotalSales DESC;
"""

country_sales_df = pd.read_sql_query(query_country_sales, conn)
conn.close()

# Plot: Bar chart of total sales by country
plt.figure(figsize=(10, 6))
plt.bar(country_sales_df['Country'], country_sales_df['TotalSales'], color='skyblue', edgecolor='black')
plt.xticks(rotation=45, ha='right')
plt.xlabel("Country")
plt.ylabel("Total Sales")
plt.title("Total Sales by Country")
plt.tight_layout()

# Save as PNG
plt.savefig("total_sales_by_country.png", dpi=300)
plt.show()

print("Chart saved as total_sales_by_country.png")
